# Titanic -- Testing out a minimal entry from Kaggle that scored high.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pprint
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
try:
    for dirname, _, filenames in os.walk('../kaggle/input'):
        for filename in filenames:
            print(os.path.join(dirname, filename))
except:
    for dirname, _, filenames in os.walk('../kaggle/input'):
        for filename in filenames:
            print(os.path.join(dirname, filename))

try:
    for dirname, _, filenames in os.walk('../kaggle/working'):
        for filename in filenames:
            print(os.path.join(dirname, filename))
except:
    for dirname, _, filenames in os.walk('../kaggle/working'):
        for filename in filenames:
            print(os.path.join(dirname, filename))
       

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

../kaggle/input/titanic/gender_submission.csv
../kaggle/input/titanic/test.csv
../kaggle/input/titanic/train.csv
../kaggle/working/train_data_binned_onehot.csv
../kaggle/working/test_data_binned_onehot.csv
../kaggle/working/submission_data.csv


In [2]:
train_data = pd.read_csv("../kaggle/input/titanic/train.csv")
test_data = pd.read_csv("../kaggle/input/titanic/test.csv")
gender_submission = pd.read_csv("../kaggle/input/titanic/gender_submission.csv")

In [20]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [23]:
# Similar to sample code on Kaggle
features = ["Sex","Parch","SibSp","Pclass"]
y = train_data["Survived"]
X = train_data[features]
X = pd.get_dummies(X, columns=features)
X["Parch_9"] = 0
colorder = list(X.columns)
colorder.sort()
X = X[colorder]
params = {
    "n_estimators": [100, 200, 500, 1000],
    "criterion": ["gini", "entropy", "log_loss"],
    "min_samples_split": [2,4,6],
    "max_features": ["sqrt","log2"],
    "n_jobs": [-1],
    "bootstrap": [True, False]
}
rf = RandomForestClassifier()
grid = GridSearchCV(rf, params, refit=True)
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.20)
grid.fit(xtrain,ytrain)
    
    


GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'bootstrap': [True, False],
                         'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_split': [2, 4, 6],
                         'n_estimators': [100, 200, 500, 1000],
                         'n_jobs': [10]})

In [24]:
grid.best_estimator_

RandomForestClassifier(criterion='entropy', max_features='log2', n_jobs=10)

In [25]:
grid.best_params_

{'bootstrap': True,
 'criterion': 'entropy',
 'max_features': 'log2',
 'min_samples_split': 2,
 'n_estimators': 100,
 'n_jobs': 10}

In [26]:
yhat = grid.predict(xtest)

In [27]:
grid.score(xtest,ytest)

0.776536312849162

In [28]:
f1_score(ytest,yhat)

0.6825396825396824

In [30]:
test_data.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [32]:
# Similar to sample code on Kaggle
features = ["Sex","Parch","SibSp","Pclass"]
X = test_data[features]
X = pd.get_dummies(X, columns=features)
colorder = list(X.columns)
colorder.sort()
X = X[colorder]
yhat = grid.predict(X)
submission = pd.DataFrame(test_data["PassengerId"])
submission["Survived"] = yhat
submission.head()
    

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [34]:
# Write the submission file 
submission.to_csv("submission.csv", index=False)